In [4]:
import torch.nn as nn
import torch.nn.functional as F

import os
import cv2

import pandas as pd
from PIL import Image
from typing import Any, Tuple, Optional, Callable
from torch.utils.data import Dataset

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.transforms import transforms

In [5]:
def read_csv(path: str) -> pd.DataFrame:
    '''
    Read a csv file.

    Args:
        path (str): Path to the csv file.

    Returns:
        pd.DataFrame: Dataframe with the csv file data.
    '''

    assert os.path.exists(path), f'CSV file not found: {path}!'
    assert os.path.splitext(path)[-1] == '.csv', f'Unsupported file type {os.path.splitext(path)[-1]}!'
    return pd.read_csv(path)

class ImageDataset(Dataset):
    def __init__(self, dataframe: pd.DataFrame, images_folder: str = './images', transform: Optional[Callable] = None, target_transform: Optional[Callable] = None) -> None:
        '''
        Image dataset.

        Args:
            dataframe (pd.DataFrame): Dataframe with the image filenames and labels.
            images_folder (str): Directory with all the images.
            transform (callable, optional): Optional transform to be applied on a sample.
            target_transform (callable, optional): Optional transform to be applied on a target.
        '''
        
        assert 'Filename' in dataframe.columns, f'Filename column not found!'
        assert os.path.exists(images_folder), f'Image folder not found: {images_folder}!'

        self.dataframe = dataframe
        self.images_folder = images_folder
        self.transform = transform
        self.target_transform = target_transform

        data = []
        targets = []

        for i, sample in dataframe.iterrows():
            # print(i)
            image = cv2.imread(os.path.join(images_folder, sample['Filename']))
            data.append(image)

            targets.append(int(sample['Label']) if 'Label' in sample else -1)

        self.data = data
        self.targets = targets

    def __len__(self) -> int:
        '''
        Returns:
            int: Length of the dataset.
        '''
        return len(self.data)

    def __getitem__(self, index: int) -> Tuple[Any, Any]:
        '''
        Args:
            index (int): Index

        Returns:
            tuple: (image, target) where target is class_index of the target class. For the public test set, target is a class from [0, 1, 2, 3, 4, 5, 6, 7, 8]. For the private test set (before releasing the test set labels), target is -1.
        '''
        img = self.data[index]
        target = self.targets[index]

        img = Image.fromarray(img)

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target

In [6]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

public_dataframe = read_csv('assignment_7_public.csv')
public_dataset = ImageDataset(public_dataframe, transform=transform)

print('Image', type(public_dataset[0][0]), public_dataset[0][0].size) # Image <class 'PIL.Image.Image'> (28, 28)
print('Target', type(public_dataset[0][1])) # Target <class 'int'>
print('Length', len(public_dataset)) # Length 85744

Image <class 'torch.Tensor'> <built-in method size of Tensor object at 0x00000233506A6580>
Target <class 'int'>
Length 85744


In [7]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32 * 7 * 7, 128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        out = self.conv1(x)
        out = self.relu1(out)
        out = self.pool1(out)
        out = self.conv2(out)
        out = self.relu2(out)
        out = self.pool2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu3(out)
        out = self.fc2(out)
        return out

# Define hyperparameters
batch_size = 64
learning_rate = 0.001
num_epochs = 10

# Create data loader
dataloader = DataLoader(public_dataset, batch_size=batch_size, shuffle=True)

# Calculate the number of samples for training and testing
train_size = int(0.7 * len(public_dataset))
test_size = len(public_dataset) - train_size

# Split the dataset into training and testing sets
train_dataset, test_dataset = random_split(public_dataset, [train_size, test_size])

# Create DataLoaders for training and testing sets
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Create the CNN model
model = CNNModel()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, data in enumerate(train_loader):
        images, labels = data

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print training progress
        if (i+1) % 100 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_step}], Loss: {loss.item():.4f}")


Epoch [1/10], Step [100/938], Loss: 1.3583
Epoch [1/10], Step [200/938], Loss: 1.0736
Epoch [1/10], Step [300/938], Loss: 0.8087
Epoch [1/10], Step [400/938], Loss: 0.8253
Epoch [1/10], Step [500/938], Loss: 0.8133
Epoch [1/10], Step [600/938], Loss: 1.0534
Epoch [1/10], Step [700/938], Loss: 0.7458
Epoch [1/10], Step [800/938], Loss: 0.9459
Epoch [1/10], Step [900/938], Loss: 0.9610
Epoch [2/10], Step [100/938], Loss: 0.8336
Epoch [2/10], Step [200/938], Loss: 0.7920
Epoch [2/10], Step [300/938], Loss: 0.7533
Epoch [2/10], Step [400/938], Loss: 1.0859
Epoch [2/10], Step [500/938], Loss: 0.6103
Epoch [2/10], Step [600/938], Loss: 0.7981
Epoch [2/10], Step [700/938], Loss: 0.7340
Epoch [2/10], Step [800/938], Loss: 0.4789
Epoch [2/10], Step [900/938], Loss: 0.5201
Epoch [3/10], Step [100/938], Loss: 0.6741
Epoch [3/10], Step [200/938], Loss: 0.4741
Epoch [3/10], Step [300/938], Loss: 0.7125
Epoch [3/10], Step [400/938], Loss: 0.5070
Epoch [3/10], Step [500/938], Loss: 0.6131
Epoch [3/10

In [8]:
from sklearn.metrics import accuracy_score

# Assuming you have a trained model called "model"
# Assuming you have a dataloader called "test_loader"

# Set the model to evaluation mode
model.eval()

# Initialize variables for tracking predictions and labels
all_predictions = []
all_labels = []

# Disable gradient computation
with torch.no_grad():
    for images, labels in test_loader:        
        # Forward pass
        outputs = model(images)
        
        # Get predicted labels
        _, predictions = torch.max(outputs, dim=1)
        
        # Move predictions and labels to CPU for evaluation
        predictions = predictions.cpu()
        labels = labels.cpu()
        
        # Collect predictions and labels
        all_predictions.extend(predictions.tolist())
        all_labels.extend(labels.tolist())

# Calculate accuracy
accuracy = accuracy_score(all_labels, all_predictions)

# Print the accuracy
print(f"Accuracy on the test set: {accuracy}")

Accuracy on the test set: 0.866778106048826


In [10]:
# export to private dataset

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

private_dataframe = read_csv('assignment_7_private.csv')
private_dataset = ImageDataset(private_dataframe, transform=transform)

print('Image', type(private_dataset[0][0]), private_dataset[0][0].size) # Image <class 'PIL.Image.Image'> (28, 28)
print('Target', type(private_dataset[0][1])) # Target <class 'int'>
print('Length', len(private_dataset)) # Length 21436

pred_loader = DataLoader(private_dataset, batch_size=batch_size, shuffle=False)

# Initialize variables for tracking predictions and labels
all_predictions = []


# Disable gradient computation
with torch.no_grad():
    for images, _ in pred_loader:        
        # Forward pass
        outputs = model(images)
        
        # Get predicted labels
        _, predictions = torch.max(outputs, dim=1)
        
        # Move predictions and labels to CPU for evaluation
        predictions = predictions.cpu()
        
        # Collect predictions and labels
        all_predictions.extend(predictions.tolist())
all_predictions

Image <class 'torch.Tensor'> <built-in method size of Tensor object at 0x000002334A399D60>
Target <class 'int'>
Length 21436


[5,
 5,
 5,
 8,
 0,
 4,
 5,
 4,
 2,
 6,
 8,
 0,
 3,
 1,
 6,
 1,
 1,
 6,
 8,
 4,
 8,
 1,
 4,
 0,
 6,
 1,
 1,
 8,
 8,
 7,
 3,
 5,
 3,
 5,
 3,
 2,
 8,
 4,
 6,
 5,
 3,
 3,
 8,
 3,
 7,
 3,
 1,
 8,
 5,
 1,
 6,
 0,
 1,
 1,
 3,
 2,
 5,
 8,
 8,
 3,
 7,
 8,
 4,
 5,
 0,
 7,
 7,
 3,
 7,
 1,
 0,
 2,
 4,
 3,
 4,
 5,
 7,
 2,
 6,
 6,
 8,
 3,
 6,
 8,
 3,
 4,
 7,
 3,
 3,
 0,
 3,
 7,
 2,
 0,
 3,
 7,
 0,
 1,
 5,
 5,
 0,
 3,
 8,
 2,
 1,
 0,
 8,
 4,
 8,
 1,
 0,
 2,
 5,
 0,
 3,
 4,
 0,
 5,
 6,
 8,
 5,
 7,
 4,
 5,
 5,
 6,
 1,
 4,
 2,
 7,
 6,
 5,
 6,
 4,
 1,
 7,
 4,
 0,
 4,
 8,
 2,
 1,
 1,
 1,
 4,
 7,
 0,
 4,
 7,
 5,
 5,
 7,
 7,
 5,
 1,
 5,
 3,
 7,
 4,
 0,
 3,
 5,
 5,
 6,
 1,
 1,
 7,
 5,
 0,
 0,
 0,
 7,
 4,
 2,
 8,
 3,
 5,
 0,
 0,
 0,
 2,
 0,
 7,
 2,
 4,
 5,
 8,
 8,
 5,
 3,
 3,
 7,
 7,
 5,
 5,
 0,
 4,
 6,
 3,
 4,
 3,
 8,
 1,
 5,
 4,
 3,
 3,
 8,
 8,
 5,
 7,
 3,
 8,
 8,
 6,
 8,
 6,
 7,
 3,
 0,
 4,
 6,
 2,
 7,
 5,
 2,
 8,
 0,
 3,
 8,
 2,
 1,
 3,
 0,
 2,
 5,
 0,
 3,
 0,
 0,
 8,
 3,
 7,
 0,
 0,
 8,
 3,
 3,
 2,
 2,


In [11]:
preds = all_predictions

submission = pd.DataFrame({'Label': preds})
submission.to_csv('assignment_7.csv', index=True, index_label='Id')